In [11]:
clip = '''
All_vertices: 41.929516810070005
V1v: 25.237121403830812
V1d: 26.72601111511618
V2v: 27.080041538796127
V2d: 25.042165432503253
V3v: 27.75684477202721
V3d: 29.784647602449006
hV4: 35.775121197319095
EBA: 51.10947146894375
FBA-1: 58.06784209856548
FBA-2: 52.41167240426183
mTL-bodies: nan
OFA: 46.32456842713712
FFA-1: 57.199066745763595
FFA-2: 54.41529085392554
mTL-faces: nan
aTL-faces: nan
OPA: 49.66800920015078
PPA: 56.89619587342665
RSC: 42.84693273603553
OWFA: 37.82385482328867
VWFA-1: 48.46111953948095
VWFA-2: 48.91085496329604
mfs-words: 51.5281072471792
mTL-words: 56.74245319590875
early: 24.81091789025866
midventral: 38.601622571610925
midlateral: 42.701906269921366
midparietal: 39.80758394389576
ventral: 51.10961787026301
lateral: 50.27785129294519
parietal: 44.713287004207274
'''.replace(":", "").split()

In [12]:
sam = '''
All_vertices: 39.81450071009121
V1v: 44.77889276003072
V1d: 45.158985468331615
V2v: 47.01335881261364
V2d: 42.583003443863106
V3v: 44.319974136360834
V3d: 44.657488297395844
hV4: 42.94722114416429
EBA: 35.02657262895985
FBA-1: 34.56240963518321
FBA-2: 30.565532995054006
mTL-bodies: nan
OFA: 41.45353491855278
FFA-1: 42.12339943448707
FFA-2: 36.36046442132052
mTL-faces: nan
aTL-faces: nan
OPA: 46.405565245353266
PPA: 49.34543786431301
RSC: 40.55368737295301
OWFA: 37.674354611276996
VWFA-1: 32.675814118388644
VWFA-2: 25.532729388241346
mfs-words: 33.49780221310939
mTL-words: 38.221436020411666
early: 41.53820859707954
midventral: 42.27753267770613
midlateral: 42.80661783506606
midparietal: 46.03186606478772
ventral: 39.12688111437165
lateral: 33.74752494246857
parietal: 41.24233285603383
'''.replace(":", "").split()

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

rois_score = []

for i in range(2, len(sam), 2):
    if np.isnan(float(sam[i+1])):
        continue
    
    roi = sam[i]
    
    if roi in ["V1v", "V1d", "V2v", "V2d", "V3v", "V3d", "hV4"]:
        roi_class = 'prf-visualrois'
    elif roi in ["EBA", "FBA-1", "FBA-2", "mTL-bodies"]:
        roi_class = 'floc-bodies'
    elif roi in ["OFA", "FFA-1", "FFA-2", "mTL-faces", "aTL-faces"]:
        roi_class = 'floc-faces'
    elif roi in ["OPA", "PPA", "RSC"]:
        roi_class = 'floc-places'
    elif roi in ["OWFA", "VWFA-1", "VWFA-2", "mfs-words", "mTL-words"]:
        roi_class = 'floc-words'
    elif roi in ["early", "midventral", "midlateral", "midparietal", "ventral", "lateral", "parietal"]:
        roi_class = 'streams'
    
    rois_score.append({
        "ROI": sam[i],
        "ROI class": roi_class,
        "CLIP": float(clip[i+1]),
#         "CLIP text": float(text_only[i+1]),
        "SAM": float(sam[i+1])
    })
    
df = pd.DataFrame(rois_score)

df.head()

,ROI,ROI class,CLIP,SAM
0,V1v,prf-visualrois,25.237121,44.778893
1,V1d,prf-visualrois,26.726011,45.158985
2,V2v,prf-visualrois,27.080042,47.013359
3,V2d,prf-visualrois,25.042165,42.583003
4,V3v,prf-visualrois,27.756845,44.319974


In [14]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    x=df["ROI"],
    y=df["CLIP"],
    name='CLIP', # name used in legend and hover labels
    marker_color='#EB89B5',
    opacity=0.75
))

fig.add_trace(go.Bar(
    x=df["ROI"],
    y=df["SAM"],
    name='SAM', # name used in legend and hover labels
    marker_color='#628E21',
    opacity=0.75
))

fig.update_layout(
    xaxis_title_text='ROIs', # xaxis label
    yaxis_title_text='Value', # yaxis label
    bargap=0.3, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.show()

In [15]:
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=df["ROI class"],
    y=df["CLIP"],
    histfunc='avg',
    name='CLIP', # name used in legend and hover labels
    marker_color='#EB89B5',
    opacity=0.75
))

fig.add_trace(go.Histogram(
    x=df["ROI class"],
    y=df["SAM"],
    histfunc='avg',
    name='SAM', # name used in legend and hover labels
    marker_color='#628E21',
    opacity=0.75
))

fig.update_layout(
    xaxis_title_text='ROI classes', # xaxis label
    yaxis_title_text='Value', # yaxis label
    bargap=0.3, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.show()

In [17]:
d = {}
score = 0.0
for idx, row in df.iterrows():
    d[row['ROI']] = np.argmax([
        row["CLIP"], row["SAM"]
    ])
    score += np.max([
        row["CLIP"], row["SAM"]
    ])
#     print(row['ROI'], np.argmax([
#         row["CLIP image"], row["CLIP text"], row["SAM"]
#     ]))
score/len(df)

47.95069089222068

In [18]:
d

{'V1v': 1,
 'V1d': 1,
 'V2v': 1,
 'V2d': 1,
 'V3v': 1,
 'V3d': 1,
 'hV4': 1,
 'EBA': 0,
 'FBA-1': 0,
 'FBA-2': 0,
 'OFA': 0,
 'FFA-1': 0,
 'FFA-2': 0,
 'OPA': 0,
 'PPA': 0,
 'RSC': 0,
 'OWFA': 0,
 'VWFA-1': 0,
 'VWFA-2': 0,
 'mfs-words': 0,
 'mTL-words': 0,
 'early': 1,
 'midventral': 1,
 'midlateral': 1,
 'midparietal': 1,
 'ventral': 0,
 'lateral': 0,
 'parietal': 0}